In [1]:
import tools
data = tools.get_data(16)

In [2]:
import itertools

In [29]:
def make_pattern(base_pattern, i):
    for v in base_pattern:
        yield from [v] * (i + 1)
        
def compute_digit(input, base_pattern, i):
    s = sum(x * p for x, p in zip([0] + list(input), itertools.cycle(make_pattern(base_pattern, i))))
    return abs(s) % 10

def execute_phase(input, base_pattern=[0, 1, 0, -1]):
    if isinstance(input, str):
        digits = [int(c) for c in str(input)]
    else:
        digits = input
    return [compute_digit(digits, base_pattern, i) for i in range(len(digits))]

In [4]:
def fft(input, num_phases):
    for _ in range(num_phases):
        input = execute_phase(input)
    return input

In [30]:
compute_digit([1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 0, -1], 2)

2

In [31]:
fft('12345678', 1)

[4, 8, 2, 2, 6, 1, 5, 8]

In [34]:
int(''.join(str(c) for c in fft(data.strip(), 100)[:8]))

84487724

In [35]:
data

'59750530221324194853012320069589312027523989854830232144164799228029162830477472078089790749906142587998642764059439173975199276254972017316624772614925079238407309384923979338502430726930592959991878698412537971672558832588540600963437409230550897544434635267172603132396722812334366528344715912756154006039512272491073906389218927420387151599044435060075148142946789007756800733869891008058075303490106699737554949348715600795187032293436328810969288892220127730287766004467730818489269295982526297430971411865028098708555709525646237713045259603175397623654950719275982134690893685598734136409536436003548128411943963263336042840301380655801969822\n'

In [36]:
len(data)

651

In [38]:
offset = 5975053

In [57]:
def only_back_half_fft(digits):
    half = len(digits) // 2 + (len(digits) % 2)
    front_half = digits[:half]
    back_half = digits[half:]
    new_back_half = []
    s = 0
    for v in reversed(back_half):
        s += v
        new_back_half.append(s % 10)
    return front_half + list(reversed(new_back_half))

In [54]:
def full_half_ffv(digits, num_phases):
    message_offset = int_from_list(digits[:7])
    for i in range(num_phases):
        print("Running phase", i)
        digits = only_back_half_fft(digits)
    return int_from_list(digits[message_offset:message_offset + 8])

In [58]:
full_half_ffv([int(c) for c in data.strip()] * 10000, 100)

[5, 9, 7, 5, 0, 5, 3]
Running phase 0
Running phase 1
Running phase 2
Running phase 3
Running phase 4
Running phase 5
Running phase 6
Running phase 7
Running phase 8
Running phase 9
Running phase 10
Running phase 11
Running phase 12
Running phase 13
Running phase 14
Running phase 15
Running phase 16
Running phase 17
Running phase 18
Running phase 19
Running phase 20
Running phase 21
Running phase 22
Running phase 23
Running phase 24
Running phase 25
Running phase 26
Running phase 27
Running phase 28
Running phase 29
Running phase 30
Running phase 31
Running phase 32
Running phase 33
Running phase 34
Running phase 35
Running phase 36
Running phase 37
Running phase 38
Running phase 39
Running phase 40
Running phase 41
Running phase 42
Running phase 43
Running phase 44
Running phase 45
Running phase 46
Running phase 47
Running phase 48
Running phase 49
Running phase 50
Running phase 51
Running phase 52
Running phase 53
Running phase 54
Running phase 55
Running phase 56
Running phase 57
Ru

84692524

The above solution works perfectly if the offset >= half the length of the input (x 10000 or whatever).
However, it doesn't solve the problem generally; it happens that the message we're looking for is in the
back half and so we can run a drastically simplified computation.

Below is an attempt at a more general solution using dynamic programming, although it is _much_ slower.
Even though in this instance it theoretically needs to do many fewer partial computations because it never
computes unnecessary sums, the functional overhead in figuring out which digits to compute and caching
them is substantially high, with a large enough multiplier (the worst case of the algorithm is N^2 and N
is very large in the problem, almost 1E6) that it is still much slower to compute than the naive solution.

A better solution would be to special-case computations down to some constant fraction of N, and then only use
the general computation for very early digits.

Another possible solution area would be to construct the transform as a matrix, which would happen to be an upper-right matrix. I don't know that this would end up being fruitful for a general solution, because the phase matricies would still have N^2 digits. However, it does make it more obvious that a simple solution exists for digits > N/2.

In [49]:
def int_from_list(l):
    print(l)
    return int(''.join(str(c) for c in l))

In [89]:
def compute_digit(input, final_digit, phase, cache=None):
    if cache is None:
        cache = {(i, 0): digit for i, digit in enumerate(input)}
    
    if (final_digit, phase) not in cache:
        required_positive_digits = itertools.chain.from_iterable(
            range(a + final_digit, min(a + 2 * final_digit, len(input)))
            for a in range(0, len(input), 4 * final_digit)
        )
        required_negative_digits = itertools.chain.from_iterable(
            range(a + 3 * final_digit, min(a + 4 * final_digit, len(input)))
            for a in range(0, len(input), 4 * final_digit)
        )
        required_positive_digits = list(required_positive_digits)
        positive = sum(compute_digit(input, digit, phase - 1, cache)[0] for digit in required_positive_digits)
        negative = sum(compute_digit(input, digit, phase - 1, cache)[0] for digit in required_negative_digits)
        cache[(final_digit, phase)] = abs(positive - negative) % 10
        
    return cache[(final_digit, phase)], cache

In [86]:
def positive_digits(input, final_digit):
    return list(itertools.chain.from_iterable(
        range(a + final_digit, min(a + 2 * final_digit, len(input)))
        for a in range(0, len(input), 4 * final_digit)
    ))

positive_digits([0] * 50, 45)

[45, 46, 47, 48, 49]

In [90]:
v, cache = compute_digit([int(c) for c in data.strip()] * 10000, int(data[:7]), 100)

KeyboardInterrupt: 

In [75]:
''.join(
    str(compute_digit(
        [int(c) for c in data.strip()] * 10000,
        int(data[:7]) + i,
        100,
    ))
    for i in range(8)
)

0